In [3]:
import yfinance as yf
import pandas as pd
import datetime as dt
import numpy as np

In [4]:
stocks = ["TCS.NS"]
start_date = dt.datetime.now() - dt.timedelta(days=30)
end_date = dt.datetime.now()
data = {}
for stock in stocks:
    data[stock] = yf.download(stock, start=start_date, end=end_date, interval="15m")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


## Moving Average Convergence Divergence (MACD)

In [5]:
def MACD(df, a=12, b=26, c=9):
    df = df.copy()
    df['EMA12'] = df['Close'].ewm(span=a, min_periods=a).mean()
    df['EMA26'] = df['Close'].ewm(span=b, min_periods=b).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['Signal'] = df['MACD'].ewm(span=c, min_periods=c).mean()
    return df.loc[:, ['MACD', 'Signal']]

In [6]:
for ticker in data:
    data[ticker][['MACD', 'Signal']] = MACD(data[ticker])

In [7]:
for stock, df in data.items():
    # Convert datetime index to string (preserving timezone info)
    df.index = df.index.astype(str)
    data[stock] = df

with pd.ExcelWriter("sheets/macd_signals.xlsx", engine="openpyxl") as writer:
    for stock, df in data.items():
        # Optional: write only relevant columns if needed
        # df = df[['MACD', 'Signal']] 
        df.to_excel(writer, sheet_name=stock)


## Average True Range (ATR)

In [8]:
def ATR(df, period=14):
    df = df.copy()
    df['TR'] = pd.concat([
        df['High'] - df['Low'],
        (df['High'] - df['Close'].shift(1)).abs(),
        (df['Low'] - df['Close'].shift(1)).abs()
    ], axis=1).max(axis=1, skipna=False)
    df['ATR'] = df['TR'].ewm(com=period).mean()
    return df[['ATR']]

In [9]:
for ticker in data:
    data[ticker][['ATR']] = ATR(data[ticker])

In [10]:
with pd.ExcelWriter("sheets/atr_signals.xlsx", engine="openpyxl") as writer:
    for stock, df in data.items():
        # Optional: write only relevant columns if needed
        # df = df[['MACD', 'Signal']] 
        df.to_excel(writer, sheet_name=stock)

## Bollinger Bands

In [11]:
def Boll_Bands(df, period=14, num_std_dev=2):
    df = df.copy()
    df['MA'] = df['Close'].rolling(window=period).mean()
    df['STD'] = df['Close'].rolling(window=period).std(ddof=0)  # Use ddof=0 for population std deviation
    df['Upper Band'] = df['MA'] + (num_std_dev * df['STD'])
    df['Lower Band'] = df['MA'] - (num_std_dev * df['STD'])
    df['Bandwidth'] = df['Upper Band'] - df['Lower Band']
    return df[['Upper Band', 'Lower Band', 'Bandwidth']]

In [12]:
for ticker in data:
    data[ticker][['Upper Band', 'Lower Band', 'Bandwidth']] = Boll_Bands(data[ticker])

In [13]:
with pd.ExcelWriter("sheets/boll_bands.xlsx", engine="openpyxl") as writer:
    for stock, df in data.items():
        # Optional: write only relevant columns if needed
        # df = df[['MACD', 'Signal']] 
        df.to_excel(writer, sheet_name=stock)

## Relative Strength Index (RSI)

In [14]:
def RSI(df, period=14):
    df = df.copy()
    df['change'] = df['Close'] - df['Close'].shift(1)
    df['gain'] = np.where(df['change'] >= 0, df['change'], 0)
    df['loss'] = np.where(df['change'] < 0, -df['change'], 0)
    df['avg_gain'] = df['gain'].ewm(alpha = 1/period, min_periods=period).mean()
    df['avg_loss'] = df['loss'].ewm(alpha = 1/period, min_periods=period).mean()
    df['rs'] = df['avg_gain'] / df['avg_loss']
    df['RSI'] = 100 - (100 / (1 + df['rs']))
    return df[['RSI']]


In [15]:
for ticker in data:
    data[ticker]['RSI'] = RSI(data[ticker])

In [16]:
with pd.ExcelWriter("sheets/RSI.xlsx", engine="openpyxl") as writer:
    for stock, df in data.items():
        # Optional: write only relevant columns if needed
        # df = df[['MACD', 'Signal']] 
        df.to_excel(writer, sheet_name=stock)

## Average Directional Index (ADX)

In [17]:
def ADX(df, period=20):
    df = df.copy()
    df['TR'] = pd.concat([
        df['High'] - df['Low'],
        (df['High'] - df['Close'].shift(1)).abs(),
        (df['Low'] - df['Close'].shift(1)).abs()
    ], axis=1).max(axis=1, skipna=False)
    
    df['+DM'] = np.where((df['High'] - df['High'].shift(1)) > (df['Low'].shift(1) - df['Low']), 
                         np.maximum(df['High'] - df['High'].shift(1), 0), 0)
    df['-DM'] = np.where((df['Low'].shift(1) - df['Low']) > (df['High'] - df['High'].shift(1)), 
                         np.maximum(df['Low'].shift(1) - df['Low'], 0), 0)
    
    df['+DI'] = 100 * (df['+DM'].rolling(window=period).sum() / df['TR'].rolling(window=period).sum())
    df['-DI'] = 100 * (df['-DM'].rolling(window=period).sum() / df['TR'].rolling(window=period).sum())
    
    df['DX'] = 100 * (np.abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI']))
    df['ADX'] = df['DX'].rolling(window=period).mean()
    
    return df[['ADX', '+DI', '-DI']]

In [18]:
for ticker in data:
    data[ticker][['ADX', '+DI', '-DI']] = ADX(data[ticker])

In [19]:
with pd.ExcelWriter("sheets/ADX.xlsx", engine="openpyxl") as writer:
    for stock, df in data.items():
        # Optional: write only relevant columns if needed
        # df = df[['MACD', 'Signal']] 
        df.to_excel(writer, sheet_name=stock)